In [ ]:
# For colab
!pip uninstall tensorflow -y
!pip install tensorflow==2.3.1 tensorflow-quantum
!rm -rf quantum-gans
!git clone https://github.com/WiktorJ/quantum-gans
!cd quantum-gans; pip install .

In [24]:
from typing import Any

import sympy
import math
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import numpy as np
from cirq import GridQubit, ops
from qsgenerator.utils import map_to_radians
from qsgenerator.qugans import circuits
from qsgenerator.qugans.training import Trainer, CustomSchedule
from qsgenerator.phase.circuits import build_ground_state_circuit
from qsgenerator.phase.analitical import construct_hamiltonian, get_ground_state_for_g, get_theta_v, get_theta_w, get_theta_r

In [25]:
generator_layers = 2
discriminator_layers = 2
data_bus_size = 3

In [26]:
gen, gs, disc, ds, ls, data_qubits, out_qubit = circuits.build_gan_circuits(generator_layers, discriminator_layers, data_bus_size)

In [27]:
real, real_symbols = build_ground_state_circuit(qubits=data_qubits)

In [28]:
pure_gen = gen.copy()
gen.append([disc])

In [29]:
pure_real = real.copy()
real.append([disc])

In [30]:
print("REAL GROUND STATE")
print(pure_real)

REAL GROUND STATE
(0, 2): ───────X───Z───Ry(theta_r)───X─────────────@───X───────────────────────────────────@───X─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
               │                                   │                                       │
(0, 3): ───────┼─────────────────────Ry(theta_w)───X───X───Ry(theta_w)───X───Ry(theta_v)───X───X───Ry(theta_v)───X───X─────────────@───X───────────────────────────────────@───X─────────────────────────────────────────────────────────────────────────────────────────────────────
               │                                                                                                                   │                                       │
(0, 4): ───────┼─────────────────────────────────────────────────────────────────────────────────────────────────────Ry(theta_w)───X───X───Ry(theta_w)───X───Ry(

In [31]:
print("GENERATOR")
print(pure_gen)

GENERATOR
(0, 2): ───Rx(g0)───Rz(g4)───ZZ────────────────────────Rx(g11)───Rz(g15)───ZZ────────────────
                             │                                             │
(0, 3): ───Rx(g1)───Rz(g5)───ZZ^g8────────────ZZ───────Rx(g12)───Rz(g16)───ZZ^g19───ZZ───────
                                              │                                     │
(0, 4): ───Rx(g2)───Rz(g6)────────────ZZ──────ZZ^g10───Rx(g13)───Rz(g17)───ZZ───────ZZ^g21───
                                      │                                    │
(0, 5): ───Rz(l)────Rx(g3)───Rz(g7)───ZZ^g9────────────Rx(g14)───Rz(g18)───ZZ^g20────────────


In [32]:
print("DISCRIMINATOR")
print(disc)

DISCRIMINATOR
(0, 0): ───Rx(d0)───Rz(d5)────────────ZZ────────────────Rx(d14)───Rz(d19)───ZZ────────────────
                                      │                                     │
(0, 1): ───Rz(l)────Rx(d1)───Rz(d6)───ZZ^d10───ZZ───────Rx(d15)───Rz(d20)───ZZ^d24───ZZ───────
                                               │                                     │
(0, 2): ───Rx(d2)───Rz(d7)───ZZ────────────────ZZ^d12───Rx(d16)───Rz(d21)───ZZ───────ZZ^d26───
                             │                                              │
(0, 3): ───Rx(d3)───Rz(d8)───ZZ^d11───ZZ────────────────Rx(d17)───Rz(d22)───ZZ^d25───ZZ───────
                                      │                                              │
(0, 4): ───Rx(d4)───Rz(d9)────────────ZZ^d13────────────Rx(d18)───Rz(d23)────────────ZZ^d27───


In [33]:
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([0] * len(gs)) + \
                   np.random.normal(scale=eps, size=(len(gs),))
init_disc_weights = np.random.normal(size=(len(ds),))

gen_weights = tf.Variable(init_gen_weights, dtype=tf.float32)
disc_weights = tf.Variable(init_disc_weights, dtype=tf.float32)

In [34]:
learning_rate = CustomSchedule()

opt = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [35]:
g_provider = lambda: 0

In [36]:
trainer = Trainer(g_provider, 
                  data_bus_size, 
                  disc, 
                  gen, 
                  real, 
                  out_qubit, 
                  ds, 
                  gs, 
                  real_symbols, 
                  ls,
                  use_analytical_expectation=True)

In [ ]:
epochs = 50
disc_iteration = 100
gen_iteration = 2
trainer.train(disc_weights,
      gen_weights, 
      opt, 
      epochs=epochs, 
      disc_iteration=disc_iteration, 
      gen_iteration=gen_iteration)

Instructions for updating:
Use fn_output_signature instead
Epoch 0: cost = [[0.2982991]]
Prob(real classified as real):  [[0.4976718]]
Prob(fake classified as real):  [[0.7959709]]
Epoch 0: cost = [[-0.7959709]]
Prob(fake classified as real):  [[0.7959709]]
Discriminator cost:  [[0.2982991]]
Generator weights: <tf.Variable 'Variable:0' shape=(22,) dtype=float32, numpy=
array([-10.1538925,  -5.8757625,  -5.8703175,   3.001207 , -13.019004 ,
       -12.693356 ,   1.9784341, -13.137627 , -13.786987 , -13.515038 ,
       -13.311686 , -12.854742 ,   2.3864002,   5.4604774,  -5.8757343,
       -12.866825 , -13.503126 ,  -5.8818154,  -5.876974 ,  -5.8876247,
       -13.543596 , -13.511531 ], dtype=float32)>
Discriminator weights <tf.Variable 'Variable:0' shape=(28,) dtype=float32, numpy=
array([  28.310925  ,   -1.3881841 ,   -0.61370814, -102.06886   ,
        -83.644295  ,  -14.386199  ,   50.432297  ,   52.792294  ,
         82.30965   ,  -98.54453   ,   94.65001   ,   32.676636  ,
       

In [41]:
g = 0
size = 3
ground_state = get_ground_state_for_g(g, size)
rad = map_to_radians(g)


trained_disc_weights = tf.Variable(np.array([ 2.86846906e-01,  2.07442139e+02, -1.32505005e+02,  3.37965851e+02,
        1.44702286e+02,  1.67083755e+02,  3.51270142e+02,  4.46218109e+02,
        4.07063354e+02,  1.02569595e+02,  1.62362915e+02,  1.63151154e+02,
        2.18758965e+01, -8.55859451e+01,  1.11934540e+02, -2.94222107e+01,
       -2.17204269e+02,  1.34002686e+02, -1.02429161e+02,  3.34117676e+02,
        1.70663238e-01, -7.37496872e+01,  2.35848450e+02,  5.64170044e+02,
        1.03939911e+02,  1.88091278e+02,  2.74167786e+02,  2.02242035e+02]), dtype=tf.float32)

trained_gen_weights = tf.Variable(np.array([ -54.72766  , -152.1226   , -170.0641   ,  -49.894302 ,
        -23.091322 ,  -33.009    ,  -37.506134 , -152.80588  ,
       -110.990685 ,  -72.17331  , -101.36898  ,    3.7497063,
        -75.12613  ,  -85.305084 , -182.64853  ,  -70.30415  ,
       -133.69072  , -203.25133  , -193.7694   ,  -45.40937  ,
        -32.63785  , -155.71288, rad]), dtype=tf.float32)




state_vector = tfq.layers.State()(pure_gen, symbol_names=gs + (ls,), symbol_values=tf.reshape(trained_gen_weights, (1, trained_gen_weights.shape[0]))).values.numpy()#%%

traced_out_state = cirq.partial_trace_of_state_vector_as_mixture(state_vector, [0,1,2])

In [13]:
prob, highest_prob_state = max(traced_out_state)
print(f"Max prob state has prob: {prob}")

Max prob state has prob: 0.9782552719116211


In [14]:
highest_prob_state

array([-0.17060323+0.j        , -0.53335905-0.23701504j,
        0.00803608+0.07157209j, -0.04239307+0.5933282j ,
        0.09452149+0.03919996j,  0.243991  +0.25223273j,
        0.01146544+0.0438571j ,  0.02175612+0.3675211j ], dtype=complex64)

In [15]:
get_ground_state_for_g(g, size)

array([1., 0., 0., 0., 0., 0., 0., 0.])

In [16]:
np.arccos(cirq.fidelity(highest_prob_state, get_ground_state_for_g(g,size)))

1.5416867535962475

In [17]:
cirq.fidelity(highest_prob_state, get_ground_state_for_g(g, size))

0.029105462289651607

In [166]:
trained_disc_weights = tf.Variable(np.array([ 2.86846906e-01,  2.07442139e+02, -1.32505005e+02,  3.37965851e+02,
        1.44702286e+02,  1.67083755e+02,  3.51270142e+02,  4.46218109e+02,
        4.07063354e+02,  1.02569595e+02,  1.62362915e+02,  1.63151154e+02,
        2.18758965e+01, -8.55859451e+01,  1.11934540e+02, -2.94222107e+01,
       -2.17204269e+02,  1.34002686e+02, -1.02429161e+02,  3.34117676e+02,
        1.70663238e-01, -7.37496872e+01,  2.35848450e+02,  5.64170044e+02,
        1.03939911e+02,  1.88091278e+02,  2.74167786e+02,  2.02242035e+02]), dtype=tf.float32)

trained_disc_weights = tf.Variable(np.array([ 
        2.86846906e-01, 0, 0, 0, 0,  
        1.67083755e+02,  3.51270142,  4.46218109, 4.07063354,  1.02569595,  
        -1.362915,  1.63151154, 2.18758965, -8.55859451,  
        1.11934540e+02, -2.9422210, -2.1720426,  134.34002686, -1.0242916,  
        3.34117676e+02, 231.7066323, -7.3749687,  -2.3584845,  5.64170044, 
        -11.03939911e+02, -1.88091278,  0,  2.0224203]), dtype=tf.float32)

trained_gen_weights = tf.Variable(np.array([ 0.01764052,  0.00400157,  0.00978738,  0.02240893,  0.01867558,
       -0.00977278,  0.00950088, -0.00151357, -0.00103219,  0.00410599,
        0.00144044,  0.01454274,  0.00761038,  0.00121675,  0.00443863,
        0.00333674,  0.01494079, -0.00205158,  0.00313068, -0.00854096,
       -0.0255299 ,  0.00653619]), dtype=tf.float32)

In [167]:
trainer.real_disc_circuit_eval(trained_disc_weights)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.60052913]], dtype=float32)>

In [137]:
trainer.gen_disc_circuit_eval(trained_gen_weights, trained_disc_weights)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.6005291]], dtype=float32)>